In [1]:
import numpy as np
import sys
sys.path.append('../corv/src')

import matplotlib.pyplot as plt
import matplotlib.colors as colors
from astropy.io import fits
from tqdm import tqdm
import pandas as pd

from astropy.table import Table, vstack, join
from astropy import units as u
from astropy.coordinates import SkyCoord

### Query
from astroquery.sdss import SDSS
from astroquery.gaia import Gaia
from astropy import constants as c


In [2]:
"""
Load in the el-Badry dataset
"""

catalog = Table.read('data/elbadry.fits.gz')
catalog = catalog[catalog['binary_type'] == b'WDMS']

print(len(catalog))

22563


In [3]:
print(len(catalog[catalog['R_chance_align'] < 0.01]))

14685


In [ ]:
def wd_sep(bp_rp):
    return 3.25*bp_rp + 9.625

wd_obj = []
ms_obj = []
drop = []

for i in tqdm (range(len(catalog))):
    mg1 = wd_sep(catalog['bp_rp1'][i])
    mg2 = wd_sep(catalog['bp_rp2'][i])
    
    M1 = catalog['phot_g_mean_mag1'][i] + 5 * (np.log10(catalog['parallax1'][i] / 100))
    M2 = catalog['phot_g_mean_mag2'][i] + 5 * (np.log10(catalog['parallax2'][i] / 100))
    
    if M1 > mg1 and M2 < mg2:
        wd_obj.append(1)
        ms_obj.append(2)
    elif M2 > mg2 and M1 < mg1:
        wd_obj.append(2)
        ms_obj.append(1)
    else:
        drop.append(i)
               
catalog.remove_rows(drop)
catalog['wd_obj'] = wd_obj
catalog['ms_obj'] = ms_obj

In [ ]:
def separate(catalog, column, ms_obj, wd_obj, newname = ''):      
    mstemp_arr = [ catalog[str(column) + str(ms_obj[i])][i] for i in range(len(ms_obj)) ]
    wdtemp_arr = [ catalog[str(column) + str(wd_obj[i])][i] for i in range(len(wd_obj)) ]
    
    catalog['ms_' + str(column)] = mstemp_arr
    catalog['wd_' + str(column)] = wdtemp_arr
    return catalog

### ---

convert_cols = ['source_id', 'parallax', 'parallax_over_error', 'phot_g_mean_mag', 'phot_g_mean_flux',
                'phot_g_mean_flux_error', 'phot_bp_mean_mag', 'phot_bp_mean_flux', 'phot_bp_mean_flux_error',
                'phot_rp_mean_mag', 'phot_rp_mean_flux', 'phot_rp_mean_flux_error', 'bp_rp', 'ra', 'ra_error', 'dec', 'dec_error', 'l', 'b']

for col in convert_cols:
    catalog = separate(catalog, col, ms_obj, wd_obj)
    
catalog['wd_m_g'] = catalog['wd_phot_g_mean_mag'] + 5 * np.log10(catalog['wd_parallax'] / 100)
catalog['ms_m_g'] = catalog['ms_phot_g_mean_mag'] + 5 * np.log10(catalog['ms_parallax'] / 100)

In [ ]:
plt.figure(figsize=(10,10))

plt.scatter(catalog['ms_bp_rp'], catalog['ms_m_g'], label='Main Sequence', alpha = 0.5, s=5)
plt.scatter(catalog['wd_bp_rp'], catalog['wd_m_g'], label='White Dwarf', alpha = 0.5, s=5)

plt.ylabel(r'$M_G$')
plt.xlabel(r'bp-rp')
plt.title(r'CMD')
plt.gca().invert_yaxis()
xmin, xmax = plt.xlim()

plt.plot(np.linspace(xmin, xmax, num=100), wd_sep(np.linspace(xmin, xmax, num=100)))

#plt.savefig('plots/selection/wd_ms_cmd.png')

plt.legend()
plt.show()

In [ ]:
catalog.write('data/00_base_catalog.fits', overwrite = True)

In [ ]:
print(catalog[catalog['wd_source_id'] == 3630648387747801088][['wd_source_id', 'wd_ra', 'wd_dec', 'ms_source_id', 'ms_ra', 'ms_dec']])
print(catalog[catalog['wd_source_id'] == 3053811568108421504][['wd_source_id', 'wd_ra', 'wd_dec', 'ms_source_id', 'ms_ra', 'ms_dec']])

print(np.where(catalog['wd_source_id'] == 3630648387747801088))
print(np.where(catalog['wd_source_id'] == 3053811568108421504))

gemini = catalog[[4484, 1554]]

In [ ]:
QUERY = """select source_id, radial_velocity, radial_velocity_error
    from gaiadr3.gaia_source
    where source_id in {}""".format(tuple(gemini['ms_source_id']))
rvs = Gaia.launch_job(QUERY,dump_to_file=False).get_results()
rvs.rename_column('source_id', 'ms_source_id')

gemini = join(gemini, rvs, keys='ms_source_id')

In [ ]:
gemini.write('data/gemini.fits', overwrite=True)

In [ ]:
from astropy.table import Table, vstack, hstack

only_rvs = True

stardats = []
iters = len(catalog) // 100

# MS Gaia Sources

for i in tqdm(range(iters)):
        ADQL_CODE1 = """SELECT sdss.original_ext_source_id as ms_bestobjid, gaia.source_id as ms_source_id, gaia.radial_velocity, gaia.radial_velocity_error
        FROM gaiadr3.gaia_source as gaia
        JOIN gaiaedr3.sdssdr13_best_neighbour as sdss
        ON gaia.source_id = sdss.source_id      
        WHERE gaia.source_id in {} AND
        radial_velocity IS NOT null AND
        radial_velocity_error < 7""".format(tuple(catalog['ms_source_id'][100*i:100*i+100]))
        stardats.append(Gaia.launch_job(ADQL_CODE1,dump_to_file=False).get_results())
        
gaia_d1 = vstack(stardats)

# MS SDSS4 Sources

for i in tqdm(range(iters)):
    ADQL_CODE1 = """SELECT sdss.original_ext_source_id as ms_bestobjid, gaia.source_id as ms_source_id, gaia.radial_velocity, gaia.radial_velocity_error
    FROM gaiadr3.gaia_source as gaia
    JOIN gaiaedr3.sdssdr13_best_neighbour as sdss
    ON gaia.source_id = sdss.source_id      
    WHERE gaia.source_id in {} AND
    radial_velocity IS null""".format(tuple(catalog['ms_source_id'][100*i:100*i+100]))
    stardats.append(Gaia.launch_job(ADQL_CODE1,dump_to_file=False).get_results())

gd1 = vstack(stardats)

stardats = []
iters = len(gd1) // 100

for i in tqdm(range(iters)):
    SDSS_QUERY = """select sp.bestObjID as ms_bestobjid, sp.plate as ms_plate, sp.mjd as ms_mjd, sp.fiberID as ms_fiberid, sp.instrument as ms_instrument
        from dbo.SpecObjAll as sp
        where sp.bestObjID in {}""".format(tuple(gd1['ms_bestobjid'][100*i:100*i+100]))
    
    f = SDSS.query_sql(SDSS_QUERY)
    if f is not None:
        stardats.append(SDSS.query_sql(SDSS_QUERY))

sdss_d1 = vstack(stardats)
try:
    sdss_d1 = join(sdss_d1, gd1, keys = 'ms_bestobjid')
except:
    sdss_d1.rename_column('bestobjid', 'ms_bestobjid')
    sdss_d1 = join(sdss_d1, gd1, keys = 'ms_bestobjid')

In [ ]:
gaia_d1['ms_instrument'] = ['gaia' for i in range(len(gaia_d1))]

In [ ]:
gaia_d1

In [ ]:
sdss_d1

In [ ]:
from astropy.table import Table, vstack, hstack, unique

stardats = []
iters = len(catalog) // 100

for i in tqdm(range(iters)):
    ADQL_CODE1 = """SELECT sdss.original_ext_source_id as wd_bestobjid, gaia.source_id as wd_source_id
    FROM gaiadr3.gaia_source as gaia
    JOIN gaiaedr3.sdssdr13_best_neighbour as sdss
    ON gaia.source_id = sdss.source_id      
    WHERE gaia.source_id in {}""".format(tuple(catalog['wd_source_id'][100*i:100*i+100]))
    stardats.append(Gaia.launch_job(ADQL_CODE1,dump_to_file=False).get_results())
    
gd2 = vstack(stardats)

stardats = []
iters = len(gd2) // 100

for i in tqdm(range(iters)):
    SDSS_QUERY = """select sp.bestObjID as wd_bestobjid, sp.plate as wd_plate, sp.mjd as wd_mjd, sp.fiberID as wd_fiberid, sp.instrument as wd_instrument
        from dbo.SpecObjAll as sp
        where sp.bestObjID in {}""".format(tuple(gd2['wd_bestobjid'][100*i:100*i+100]))
    try:
        f = SDSS.query_sql(SDSS_QUERY)
        if f is not None:
            stardats.append(SDSS.query_sql(SDSS_QUERY))
    except:
        pass
d2 = vstack(stardats)
d2 = join(d2, gd2, keys = 'wd_bestobjid')
d2 = unique(d2, keys = 'wd_bestobjid')

In [ ]:
d2

In [ ]:
try:
    sdss_d1.rename_column('asplate', 'ms_plate')
    sdss_d1.rename_column('asmjd', 'ms_mjd')
    sdss_d1.rename_column('asfiberid', 'ms_fiberid')
    sdss_d1.rename_column('asinstrument', 'ms_instrument')
except:
    pass

In [ ]:
sdss_d1

In [ ]:
from astropy.table import Table, Column, MaskedColumn, join, unique


In [ ]:
data = join(catalog, gaia_d1, keys = 'ms_source_id')
ms_gaia_wd_sdss4 = join(data, d2, keys = 'wd_source_id', join_type = 'inner')

data = join(catalog, sdss_d1, keys = 'ms_source_id')
ms_sdss4_wd_sdss4 = join(data, d2, keys = 'wd_source_id', join_type = 'inner')

ms_sdss5_wd_sdss4 = join(catalog, d2, keys = 'wd_source_id')

In [ ]:
ms_gaia_wd_sdss4['wd_filepath'] = ['' for i in range(len(ms_gaia_wd_sdss4))]
ms_gaia_wd_sdss4['ms_filepath'] = ['' for i in range(len(ms_gaia_wd_sdss4))]
ms_gaia_wd_sdss4['uses_sdss5'] = np.zeros(len(ms_gaia_wd_sdss4))
ms_gaia_wd_sdss4['wd_rv_from'] = ['sdss4' for i in range(len(ms_gaia_wd_sdss4))]
ms_gaia_wd_sdss4['ms_rv_from'] = ['gaia' for i in range(len(ms_gaia_wd_sdss4))]
ms_gaia_wd_sdss4['wd_rv'] = 999*np.ones(len(ms_gaia_wd_sdss4))
ms_gaia_wd_sdss4['wd_e_rv'] = 999*np.ones(len(ms_gaia_wd_sdss4))

ms_sdss4_wd_sdss4['wd_filepath'] = ['' for i in range(len(ms_sdss4_wd_sdss4))]
ms_sdss4_wd_sdss4['ms_filepath'] = ['' for i in range(len(ms_sdss4_wd_sdss4))]
ms_sdss4_wd_sdss4['uses_sdss5'] = np.zeros(len(ms_sdss4_wd_sdss4))
ms_sdss4_wd_sdss4['wd_rv_from'] = ['sdss4' for i in range(len(ms_sdss4_wd_sdss4))]
ms_sdss4_wd_sdss4['ms_rv_from'] = ['sdss4' for i in range(len(ms_sdss4_wd_sdss4))]
ms_sdss4_wd_sdss4['wd_rv'] = 999*np.ones(len(ms_sdss4_wd_sdss4))
ms_sdss4_wd_sdss4['wd_e_rv'] = 999*np.ones(len(ms_sdss4_wd_sdss4))

ms_gaia_wd_sdss4 = unique(ms_gaia_wd_sdss4, keys = 'wd_source_id')
ms_sdss4_wd_sdss4 = unique(ms_sdss4_wd_sdss4, keys = 'wd_source_id')

ms_gaia_wd_sdss4.write('data/ms_gaia_wd_sdss4.fits', overwrite = True)
ms_sdss4_wd_sdss4.write('data/ms_sdss4_wd_sdss4.fits', overwrite = True)

In [ ]:
ms_sdss5_wd_sdss4.write('sdss5/temp_ms_sdss5_wd_sdss4.fits', overwrite = True)

In [ ]:
ms_sdss4_wd_sdss4

In [ ]:
print(len(ms_gaia_wd_sdss4))
print(len(unique(ms_gaia_wd_sdss4, keys = 'wd_source_id')))

print(len(ms_sdss4_wd_sdss4))
print(len(unique(ms_sdss4_wd_sdss4, keys = 'wd_source_id')))

In [ ]:
print(np.median(ms_gaia_wd_sdss4['wd_m_g']))